# Usage Notes
This notebook demonstrates how to interact with the NWB files in the Rodgers 2022 dataset.

In [5]:
# NWB imports and DANDI imports
import pynwb
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient

In [6]:
# This ID specifies the object recognition dataset
dandiset_id = '000231'

# This specifies a path to an example NWB file, for a single session from subject 231CR
filepath = 'sub-231CR/sub-231CR_ses-20190921T144923_behavior+ecephys+image.nwb'

In [12]:
## This loads the NWB file without requiring the user to download the file
# First get the URI for the NWB file on dandihub
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
    s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)

# This creates an IO object for the file
# If the file was downloaded locally, replace `s3_path` with the path to the file, and don't specify a driver
io = NWBHDF5IO(s3_path, mode='r', load_namespaces=True, driver='ros3')

# Read the file
nwbfile = io.read()

In [13]:
nwbfile

root pynwb.file.NWBFile at 0x139891534850656
Fields:
  acquisition: {
    behavioral video <class 'pynwb.image.ImageSeries'>,
    extracellular array recording <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    4253 <class 'pynwb.device.Device'>,
    high-speed camera <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    all_channels <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Video of mouse performing whisker-based shape discrimination. This session also contains neural recordings in barrel cortex
  experimenter: ['Chris Rodgers']
  file_create_date: [datetime.datetime(2022, 4, 14, 18, 46, 55, 752342, tzinfo=tzoffset(None, -14400))]
  identifier: 190921_231CR
  institution: Columbia University
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  keywords: <StrDataset for HDF5 dataset "keywords": shape (12,), type "|O">
  processing: {
    behavior <class 'pynwb.ba

In [19]:
print(nwbfile.trials[:][['start_time', 'stop_time', 'response_window_open_time', 'stimulus']])

     start_time  stop_time  response_window_open_time stimulus
id                                                            
4         5.910     13.085                      8.932   convex
5        13.085     19.335                     16.105  concave
6        19.335     26.570                     22.357   convex
7        26.570     32.960                     29.591  concave
8        32.960     39.885                     35.981   convex
..          ...        ...                        ...      ...
245    1980.075   1987.075                   1983.097   convex
246    1987.075   1994.745                   1990.098   convex
247    1994.745   2010.905                   1997.767  concave
248    2010.905   2054.200                   2013.930   convex
249    2054.200   2071.970                   2057.223   convex

[246 rows x 4 columns]


In [27]:
nwbfile.processing['behavior']['processed_position_whisker_C2']['angle']

angle pynwb.base.TimeSeries at 0x139891533389888
Fields:
  comments: no comments
  continuity: continuous
  conversion: 1.0
  data: <HDF5 dataset "data": shape (412375,), type "<f8">
  description: (float) the angle of the whisker in degrees, with 0 horizontal and positive protracting toward top of frame
  interval: 1
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (412375,), type "<f8">
  timestamps_unit: seconds
  unit: degrees

In [31]:
print(nwbfile.processing['behavior']['contacts_by_whisker_C2'][:][['start_time', 'stop_time', 'angle']])

      start_time  stop_time      angle
id                                    
0          7.925      7.940  18.299733
1          7.970     10.295   8.927367
2         21.355     21.355  21.127114
3         21.405     21.415  21.502236
4         21.440     21.445  20.567538
...          ...        ...        ...
1102    2041.825   2041.835  14.718447
1103    2059.385   2059.450   4.739310
1104    2059.490   2059.515   5.581840
1105    2059.550   2059.580   4.801581
1106    2059.605   2059.630   6.810567

[1107 rows x 3 columns]


In [34]:
nwbfile.acquisition['extracellular array recording'].data[:1000000]

array([[-2551, -2293, -2906, ..., -3222, -3925, -4305],
       [-2563, -2306, -2930, ..., -3227, -3934, -4294],
       [-2593, -2321, -2935, ..., -3219, -3941, -4299],
       ...,
       [  821,  1258,  1194, ...,  1013,   930,  1141],
       [  810,  1268,  1204, ...,  1042,   901,  1105],
       [  810,  1258,  1192, ...,  1026,   906,  1115]], dtype=int16)

In [40]:
print(nwbfile.units[:][['inhibitory', 'depth', 'spike_times']])

     inhibitory   depth                                        spike_times
id                                                                        
39        False   485.0  [37.215446883180434, 62.707934376765024, 62.73...
68        False   145.0  [155.29713108782187, 194.45698095183477, 199.1...
74        False   425.0  [35.0990090533536, 124.02156464108779, 124.025...
79        False   885.0  [35.01400887276583, 177.74264544127064, 233.51...
92        False   225.0  [20.989145742806684, 24.481253161989116, 28.77...
101       False   505.0  [35.008408860868286, 46.32286623241501, 101.78...
104       False   505.0  [62.66546761987527, 62.731267759671454, 100.83...
106       False   485.0  [246.60022506647968, 262.8729596388373, 324.27...
130       False   625.0  [10.219022861063138, 10.268689633249707, 11.96...
136       False   525.0  [11.473558859730133, 12.995295426080048, 13.24...
147       False   445.0  [155.28886440359213, 221.1516376661634, 284.91...
162       False  1005.0  

In [43]:
nwbfile.acquisition['behavioral video'].external_file[0]

'sub-231CR_ses-20190921T144923_behavior+ecephys+image/f80f5af1-6140-4d88-9734-52fcfe0dca47_external_file_0.mkv'